# Simulation code

Maël Delem  
2/02/2024

> **Packages and setup**
>
> Down below is the code to load necessary packages used for the simulation and analysis, along with some setups for the whole document (*hover over the numbers on the far right for additional explanation of code and mechanics*).
>
> ``` r
>
> # ═══ Packages ═════════════════════════════════════════════════════════════════
>
> if (!require(librarian)) install.packages(librarian)
> ```
>
> Line 4  
> The package `librairian` eases package management with the “shelf” function, which automatically: (1) checks if a package is installed; (2) installs it if need be; (3) loads the package like the “library()” function would.
>
> Line 28  
> `reticulate` allows to translate and transfer objects and functions from R to Python and vice-versa, and was thus of primary importance for the successful use of the Python toolbox on our simulated data.
>
> Lines 35,51  
> These are personal custom color palettes meant to extend my favourite palette, the color-atypical friendly Okabe-Ito color palette. The palette originally has only eight colors, but I will need nine, then up to 30 for later graphs, so I extended it with a hand-picked selection of mine.
>
> Lines 55,69  
> These are R objects that were the results of a previous run of the simulation.
>
>     Le chargement a nécessité le package : librarian
>
> ``` r
>
> library(librarian)
>
> # now putting packages on our library's shelves:
> shelf(
>   # ─── data management ─────────────────
>   holodeck,       # simulating multivariate data
>   cluster,        # dissimilarity matrices
>   
>   # ─── modelling ───────────────────────
>   mclust,         # mixture clustering
>   
>   #  data visualization ──────────────
>   # palettes
>   viridis,        # colour-blind friendly palettes
>   # interactive
>   plotly,         # interactive plots
>   ggdendro,       # dendrograms
>   seriation,      # dissimilarity plots
>   webshot2,       # HTML screenshots for Word render
>   webshot,
>   
>   # ─── essential package collections ───
>   doParallel,     # parallel execution
>   easystats,      # data analysis ecosystem
>   reticulate,     # R to Python
>   tidyverse,      # modern R ecosystem
> )
>
> # ─── Global cosmetic theme ───
> theme_set(theme_modern(base_size = 14))
>
> pal_okabe_ito <- c(
>   "#E69F00", "#56B4E9", "#009E73",                            
>   "#F5C710", "#0072B2", "#D55E00", "#CC79A7", "#6c0009")      
>
> # We'll need 9 colors at some point
> pal_okabe_ito_extended <- c(                                 
>   "#E69F00", "#56B4E9", "#009E73",                           
>   "#F5C710", "#0072B2", "#D55E00", "#CC79A7", "#6c0009", "#414487FF")
>
> # We'll need 30 colors at another moment
> cool_30_colors <- c(                                                   
>   "#3d51b4", "#414487FF", "#003d73", "#440154FF", "#6c0009", "#b64e4e",
>   "#D55E00", "#E69F00", "#F5C710", "#FDE725FF", "#f2bb7b", "#f1afad", "#CC79A7", 
>   "#e57774", "#7AD151FF", "#57b571", "#318a4a", "#009E73", "#22A884FF", 
>   "#2A788EFF", "#0072B2", "#2da6b5", "#56B4E9", "#889be0", "#6677e0",   
>   "#3d51b4", "#414487FF", "#003d73", "#440154FF", "#6c0009", "#b64e4e"  
>   )
>
> path = "data/"
>
> df <- read_rds(paste0(path, "df.RDS"))
> df_embeds <- read_rds(paste0(path, "df_embeds.RDS"))
> # Categorical and visual embeddings
> df_embeds_categ  <- read_rds(paste0(path, "df_embeds_categ.RDS"))
> df_embeds_visual <- read_rds(paste0(path, "df_embeds_visual.RDS"))
> # Subject embeddings per sub-group
> df_embed_c_sub  <- read_rds(paste0(path, "df_embed_c_sub.RDS"))
> df_embed_cs_sub <- read_rds(paste0(path, "df_embed_cs_sub.RDS"))
> df_embed_v_sub  <- read_rds(paste0(path, "df_embed_v_sub.RDS"))
> df_embed_vs_sub <- read_rds(paste0(path, "df_embed_vs_sub.RDS"))
> # Accuracy of the unsupervised alignment (bad = not tidy data)
> df_accuracy_all_bad <- read_rds(paste0(path, "df_accuracy_all_bad.RDS"))
> df_accuracy_cat_bad <- read_rds(paste0(path, "df_accuracy_cat_bad.RDS"))
> # Coordinates of the aligned embeddings from the Python output
> coordinates_aligned_embeddings <- read_rds(paste0(path, "coordinates_aligned_embeddings.RDS"))
> ```

In [ ]:

# The function takes the variance and covariance of the imagery distributions
# as arguments
generate_osv_model <- function(var, cov){
  df <- 
    tibble(group = rep(c("aph", "phant"), each = 8)) |> 
    group_by(group) |> 
    mutate(
      spatial_group = c(rep("spa_low", 4), rep("spa_high", 4)),
      vis_spa_group = paste0(group, "_", spatial_group),
      verbal_group = "verbal_low",
      verbal_group  = case_when(
        vis_spa_group == "aph_spa_low" ~ "verbal_high", 
        vis_spa_group == "phant_spa_low" ~ "verbal_mid",
        TRUE ~ verbal_group)
    ) |> 
    group_by(vis_spa_group) |> 
    # ─── visual ───
    sim_discr(
      n_vars = 1, 
      var = var, 
      cov = cov, 
      # aph_s, aph_v, phant_s, phant_v
      group_means = c(0, 0, 0.6, 0.87), 
      name = "v") |> 
    # ─── spatial ───
    sim_discr(
      n_vars = 1,  
      var = var, 
      cov = cov, 
      # aph_s, aph_v, phant_s, phant_v
      group_means = c(0.75, 0.4, 0.7, 0.3), 
      name = "s") |>
    # ─── verbal ───
    sim_discr(
      n_vars = 1,  
      var = var, 
      cov = cov, 
      # aph_s, aph_v, phant_s, phant_v
      group_means = c(0.3, 0.7, 0.3, 0.5), 
      name = "i") |>
    rename(
      visual_imagery  = v_1,
      spatial_imagery = s_1,
      verbal_profile  = i_1
      )
}

df <- generate_osv_model(0.03, 0)
